From BATS: https://github.com/CompTop/BATS/blob/master/include/linalg/sparse_fact.hpp#L314

0. Ltilde initialized as identity
1. Extract scale from EL
    + for unit EL, we have EL * L = Ltil * EL
    + with diagonal row scaling, use D EL L = D Ltil D^{-1} D EL
2. construct index map
3. fill-in non-zeros of Ltilde
    + Ltilde(idx_map[i], idx_map[j]) = L[i,j]
    
Alternatively: Ltilde = E * L * EL.T(); (after extracting scaling)

In [9]:
def col_pivot(A, j):
    """
    last non-zero index of column j in A
    -1 if column is all zero
    """
    return max([i for i in range(A.shape[0]) if A[i,j] != 0], default=-1)

def index_map(A):
    """
    index_map[j] is pivot index of column j in A
    """
    index_map = []
    for j in range(A.shape[1]):
        index_map.append(col_pivot(A, j))
        
    return index_map

In [6]:
import numpy as np
A = np.eye(3)
A[0,0] = 0

In [10]:
col_pivot(A, 0)

-1

In [19]:
def EL_L_commute(EL, L):
    m, n = EL.shape
    # TODO: extract scaling, so EL only has 1s or 0s
    
    imap = index_map(EL)
    L2 = np.eye(m)
    for j in range(n):
        if (imap[j] == -1):
            break
        for i in range(j,n):
            if (imap[i] == -1):
                break
            L2[imap[i],imap[j]] = L[i,j]
            
    # TODO: apply scaling
                
                
    return L2

In [26]:
def EL_pseudoinverse(EL):
    """
    return pseudoinverse
    """
    EL2 = EL.T
    for i in range(EL2.shape[0]):
        for j in range(EL2.shape[1]):
            if EL2[i,j] != 0:
                EL2[i,j] = 1/EL2[i,j]
                
    return EL2

In [27]:
def EL_L_commute(EL, L):
    # TODO: handle scaling of EL   
    # EL.T is left-pseudoinverse of EL if you have unit entries
    # could replace EL.T with pseudo inverse to handle scaling
    return EL @ L @ EL_pseudoinverse(EL)

In [29]:
EL = np.array([[2,0],[0,3]])
L = np.array([[1,0],[2,1]])
L2 = EL_L_commute(EL,L)
L2 @ EL - EL @ L

array([[0, 0],
       [0, 0]])

In [30]:
EL = np.array([[0,0],[2,0]])
L = np.array([[1,0],[2,1]])
L2 = EL_L_commute(EL,L)
L2 @ EL - EL @ L

array([[0, 0],
       [0, 0]])